<a href="https://colab.research.google.com/github/Irisrangel/Raspagem_de_texto-web_to_json/blob/main/Estruturando_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup

In [35]:
# Link de origem
BASE_URL = "https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/pronunciamentos/pronunciamentos-do-presidente-da-republica"
# Pasta de destino para salvar os pronunciamentos em Json
DATA_DIR = "pronunciamentos"

In [36]:
response = requests.get(BASE_URL)

In [ ]:
response.text

In [38]:
bs = BeautifulSoup(response.text)

In [39]:
articles = bs.find_all("article")

In [ ]:
articles

In [41]:
len(articles)

17

In [42]:
article = articles[0]

In [ ]:
article

In [44]:
article = articles[0]
url = article.find(class_="summary url")
tags = article.find_all(rel="tag")
datetime = article.find_all(class_="summary-view-icon")

In [45]:
url

<a class="summary url" href="https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/pronunciamentos/pronunciamentos-do-presidente-da-republica/pronunciamento-do-presidente-da-republica-jair-bolsonaro-em-cadeia-de-radio-e-televisao-brasilia-df" title="Document">Pronunciamento do Senhor Presidente da República, Jair Bolsonaro, em cadeia de rádio e televisão</a>

In [46]:
url["href"]

'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/pronunciamentos/pronunciamentos-do-presidente-da-republica/pronunciamento-do-presidente-da-republica-jair-bolsonaro-em-cadeia-de-radio-e-televisao-brasilia-df'

In [47]:
tags

[<a class="link-category" href="https://www.gov.br/planalto/@@search?Subject:list=Pronunciamento" rel="nofollow tag">Pronunciamento</a>,
 <a class="link-category" href="https://www.gov.br/planalto/@@search?Subject:list=Presidente" rel="nofollow tag">Presidente</a>,
 <a class="link-category" href="https://www.gov.br/planalto/@@search?Subject:list=Nova Previdência" rel="nofollow tag">Nova Previdência</a>]

In [48]:
tags = [t.text for t in tags]
tags

['Pronunciamento', 'Presidente', 'Nova Previdência']

In [49]:
datetime

[<span class="summary-view-icon">
 <i class="icon-day"></i>
                         20/02/2019
                     </span>, <span class="summary-view-icon">
 <i class="icon-hour"></i>
                         20h53
                     </span>, <span class="summary-view-icon">
 <i class="icon-page"></i>
                         Página
                     </span>]

In [50]:
datetime[0].text.strip()

'20/02/2019'

In [51]:
date = datetime[0].text.strip()
time = datetime[1].text.strip()
(date, time)

('20/02/2019', '20h53')

In [52]:
def canonical_date(datestr):
  return "".join(reversed(datestr.split("/")))

In [53]:
canonical_date(date)

'20190220'

Organizando o código até aqui em uma classe:

In [56]:
class Pronouncement:
  def __init__(self, bs_article=None):

    self.url = bs_article.find(class_="summary url")["href"]
    self.tags = [t.text for t in bs_article.find_all(rel="tag")]
    
    datetime = article.find_all(class_="summary-view-icon")
    self.date = canonical_date(datetime[0].text.strip())
    self.time = time = datetime[1].text.strip()
  
    self.text = self._extract_text()

  def _extract_text(self):
    return "Teste"

In [57]:
p = Pronouncement(article)

In [58]:
p.url

'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/pronunciamentos/pronunciamentos-do-presidente-da-republica/pronunciamento-do-presidente-da-republica-jair-bolsonaro-em-cadeia-de-radio-e-televisao-brasilia-df'

In [59]:
p.date

'20190220'

In [61]:
p.time

'20h53'

In [62]:
p.tags

['Pronunciamento', 'Presidente', 'Nova Previdência']

In [65]:
p.text

'Teste'

In [67]:
class Pronouncement:
  def __init__(self, bs_article=None):

    self.url = bs_article.find(class_="summary url")["href"]
    self.tags = [t.text for t in bs_article.find_all(rel="tag")]
    
    datetime = article.find_all(class_="summary-view-icon")
    self.date = canonical_date(datetime[0].text.strip())
    self.time = time = datetime[1].text.strip()
  
    self.text = self._extract_text()

  def _extract_text(self):
    r = requests.get(self.url)
    bs = BeautifulSoup(r.text)
    return bs.find(id="parent-fieldname-text").text.strip()

In [68]:
p2 = Pronouncement(article)

In [69]:
p2.text

'Brasília/DF, 20 de fevereiro de 2019\n\xa0\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Boa noite. Estamos determinados a mudar o rumo do nosso País. Nosso objetivos são claros: resgatar nossa segurança, fazer a economia crescer e servir a quem realmente manda no País: a população brasileira.\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Sendo assim, ontem encaminhamos ao Congresso um pacote anticrime e hoje iniciamos o processo de criação de uma nova Previdência.\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 É fundamental equilibrarmos as contas do País para que o sistema não quebre, como já aconteceu com outros países e alguns estados brasileiros. Precisamos garantir que hoje e sempre todos receberão seus benefícios em dia e o governo tenha recursos para ampliar investimentos na melhoria de vida da população e na geração de empregos.\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 A nova Previdência será justa para todos, sem privilégios. Ricos e pobres, servidores públicos, políticos ou trabalhadores privados, tod

In [70]:
print(p2.text)

Brasília/DF, 20 de fevereiro de 2019
 
          Boa noite. Estamos determinados a mudar o rumo do nosso País. Nosso objetivos são claros: resgatar nossa segurança, fazer a economia crescer e servir a quem realmente manda no País: a população brasileira.
          Sendo assim, ontem encaminhamos ao Congresso um pacote anticrime e hoje iniciamos o processo de criação de uma nova Previdência.
          É fundamental equilibrarmos as contas do País para que o sistema não quebre, como já aconteceu com outros países e alguns estados brasileiros. Precisamos garantir que hoje e sempre todos receberão seus benefícios em dia e o governo tenha recursos para ampliar investimentos na melhoria de vida da população e na geração de empregos.
          A nova Previdência será justa para todos, sem privilégios. Ricos e pobres, servidores públicos, políticos ou trabalhadores privados, todos seguirão as mesmas regras de idade e tempo de contribuição.
          Também haverá a reforma do sistema de proteç

Agora precisamos armazenar os textos em Json para conseguir recuperar depois:

In [71]:
p.__dict__

{'date': '20190220',
 'tags': ['Pronunciamento', 'Presidente', 'Nova Previdência'],
 'text': 'Teste',
 'time': '20h53',
 'url': 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/pronunciamentos/pronunciamentos-do-presidente-da-republica/pronunciamento-do-presidente-da-republica-jair-bolsonaro-em-cadeia-de-radio-e-televisao-brasilia-df'}

In [77]:
class Pronouncement:
  def __init__(self, bs_article=None):

    if bs_article is not None:
      self.url = bs_article.find(class_="summary url")["href"]
      self.tags = [t.text for t in bs_article.find_all(rel="tag")]
      
      datetime = bs_article.find_all(class_="summary-view-icon")
      self.date = canonical_date(datetime[0].text.strip())
      self.time = time = datetime[1].text.strip()
    
      self.text = self._extract_text()

  def _extract_text(self):
    r = requests.get(self.url)
    bs = BeautifulSoup(r.text)
    return bs.find(id="parent-fieldname-text").text.strip()

  def save_tojson(p, savedir, name=None):
    if name is None:
      name = f"{p.date}-{p.time}.json"
    path = os.path.join(savedir, name)
    with open(path, "w") as jsonfile:
      json.dump(p.__dict__, jsonfile)    

  def load_fromjson(filepath):
    with open(filepath, "r") as jsonfile:
      data = json.load(jsonfile)
      p = Pronouncement()
      p.date = data["date"]
      p.time = data["time"]
      p.tags = data["tags"]
      p.url = data["url"]
      p.text = data["text"]
      return p


In [78]:
pronouncements = [Pronouncement(art) for art in articles]

In [79]:
# Salvando os pronuncimaneto em Json
for p in pronouncements:
  Pronouncement.save_tojson(p, DATA_DIR)

In [80]:
len(pronouncements)

17

Vamos agora tentar recuperar os textos a partir dos Jsons salvos.

In [81]:
os.listdir(DATA_DIR)

['20201224-20h30.json',
 '20200331-21h07.json',
 '20200901-11h33.json',
 '20200324-21h25.json',
 '20200312-22h48.json',
 '20200812-20h00.json',
 '20200416-20h03.json',
 '20200907-20h07.json',
 '20190220-20h53.json',
 '20190424-21h35.json',
 '20191224-21h43.json',
 '20200306-21h00.json',
 '20190501-21h45.json',
 '20210323-20h34.json',
 '20200408-20h35.json',
 '20190823-21h21.json',
 '20210205-14h58.json']

In [82]:
filenames = os.listdir(DATA_DIR)

In [84]:
pronunciamentos = []
for name in filenames:
  path = os.path.join(DATA_DIR, name)
  pronunciamentos.append(Pronouncement.load_fromjson(path))

In [85]:
len(pronunciamentos)

17

In [93]:
for i in range(2, 8, 2):
  print("Num:", i)
  print(pronunciamentos[i].text)
  print("===========================")

Num: 2
Palácio da Alvorada, 01 de setembro de 2020
 
Bom dia.
Acabamos uma reunião muito produtiva com os líderes e tomamos duas decisões. 
A primeira, encaminhar, na quinta-feira, a Reforma Administrativa, que fique bem claro, não atingirá nenhum dos atuais servidores. Ela se aplicará apenas os futuros servidores concursados.
 E a segunda medida, depois que nós criamos há praticamente 5 meses o Auxílio Emergencial de 600 reais, ele validou-se por 3 meses. Por Decreto nós o prorrogamos por mais 2 meses e agora resolvemos prorrogá-lo por Medida Provisória, até o final do ano. 
E o valor, com vínhamos dizendo, 600 é muito para quem paga, no caso o Brasil e podemos dizer que não é um valor o suficiente muitas vezes, para todas as necessidades, mas basicamente atende. Até porque o valor definido agora há pouco, ele é um pouco superior a 50% do valor do salário mínimo, ou melhor do Bolsa Família. Obrigado aí, do Bolsa Família. Então, nós decidimos aqui, até atendendo a economia em cima da r